In [ ]:
# import urllib.request
# from bs4 import BeautifulSoup
# import csv
# import pandas as pd
# import numpy as np
# import requests 
# import html5lib
# import urllib3

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import os

In [17]:
os.environ['PATH'] += r"/Users/marcolee/Desktop/chromedriver_mac_arm64"
driver = webdriver.Chrome()
url = "https://chooseyourstory.com/Stories/Sci-Fi.aspx"
driver.get(url)
original_window = driver.current_window_handle
driver.implicitly_wait(5)

In [3]:
def switch_window(driver,original_window):
    for window in driver.window_handles:
        if window != original_window:
            driver.switch_to.window(window)
            break

In [4]:
#get description and click play button
def start_story(driver,original_window):
    #content = driver.find_element(By.CLASS_NAME,'storygame-description').text
    play_button = driver.find_element(By.CLASS_NAME,'storygame-play-button')
    play_button.click()
    switch_window(driver,original_window)
    

In [37]:
# get the story after play button
def tree_story(driver,stories={}): #driver, dict --> text, dict[title]=story
    cur_title = driver.find_element(By.ID,'_storyTitle').text
    content = driver.find_element(By.XPATH,"/html/body/div[3]/div[1]").text + "\n"
    if not stories:
        flag = True
    else:
        flag = False
    if cur_title not in stories:
        stories[cur_title] = content
    # else:
    #     return stories, cur_title
    list = driver.find_elements(By.TAG_NAME,'ul')
    #buttons = [link.text for link in list]
    links = list[1].find_elements(By.TAG_NAME,'a')
    tree = {}
    for i in range(len(links)):
        list = driver.find_elements(By.TAG_NAME,'ul')[1]
        link = list.find_elements(By.TAG_NAME,'a')[i]
        if link.text == ">END":
            tree = ">END"
            break
        #print(link.text)
        title = link.text
        link.click()
        stories, subtree = tree_story(driver,stories)
        # content += tmp_content
        tree[title] = subtree

    #go back
    list = driver.find_element(By.TAG_NAME,'ul')
    back = list.find_element(By.TAG_NAME,'a')
    back.click()
    if flag:
        return stories, {cur_title:tree}
    return stories, tree


In [19]:
# making a list of stories
table = driver.find_element(By.ID,"MainContentPlaceHolder_StoriesGridView")
stories = table.find_elements(By.TAG_NAME,'tr')
links = []
titles = []
for story in stories[1:]:
    #new_driver = copy.copy(driver)
    link = story.find_element(By.TAG_NAME,'a')
    links.append(link.get_attribute("href"))
    titles.append(link.text)
    #switch_window(driver,original_window)
    #start_story(driver,original_window)
    
#links

In [38]:
def initialize_driver(url):
    driver = webdriver.Chrome()
    driver.get(url)
    original_window = driver.current_window_handle
    #driver.implicitly_wait(5)
    return driver,original_window

In [21]:
len(links)

109

In [39]:
scifi = []
for link,title in zip(links,titles):
    cur_driver,origin_window = initialize_driver(link)
    start_story(cur_driver,origin_window)
    stories, indexes = tree_story(cur_driver)
    scifi.append({'title':title,'stories':stories,'tree':indexes})
    cur_driver.quit()
    break

In [40]:
indexes

{'ARENA DAY': {'>CIRCLE': {'>REPAIR': {'>LOYALTY': '>END',
    '>REBELLION': '>END'}},
  '>STRIKE': {'>RESPECT': {'>TRIBUTE': {'>LATER': {'>SACRIFICE': '>END',
      '>HARVEST': '>END'}},
    '>REPAIR': '>END'},
   '>SCORN': {'>REPAIR': '>END',
    '>TRIBUTE': {'>LATER': {'>SACRIFICE': '>END', '>HARVEST': '>END'}}}}}}

In [ ]:
stories

In [42]:
cur_driver.quit()
driver.quit()

In [27]:
import json

In [43]:
scifi[0].keys()

dict_keys(['title', 'stories', 'tree'])

In [44]:
with open('scifi_text.json','w') as file:
    json.dump(scifi,file)

In [45]:
import pandas as pd

In [46]:
scifi2 = pd.read_json('scifi_text.json')
scifi2

,title,stories,tree
0,>DEVOTION,{'ARENA DAY': 'The gate opens. The assembled s...,{'ARENA DAY': {'>CIRCLE': {'>REPAIR': {'>LOYAL...


In [36]:
scifi3 = scifi2.iloc[0][1].keys()
print(scifi3)

dict_keys(['ARENA DAY', '>CIRCLE', '>REPAIR', '>LOYALTY', '>REBELLION', '>STRIKE', '>RESPECT', '>TRIBUTE', '>LATER', '>SACRIFICE', '>HARVEST', '>SCORN'])
